Assignment 3: Streaming analytics on text data

Part 1: Scraping

In [1]:
# Import packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
import datetime as dt
from datetime import timedelta
import time
import re
from urllib.parse import urlparse
import json
import os

In [2]:
#So we can work easier with posted_at/age of the post
def age_to_time(i):
    if 'minute' in i:
        return int(i.split()[0])
    elif 'hour' in i:
        return int(i.split()[0]) * 60
    elif 'day' in i:
        return int(i.split()[0]) * 1440
    else:
        return 10000 #With this we will be sure that we will throw away the var

#Here we get the title or header of the site
def contains_desired_class(class_attr):
    return class_attr and re.search(r'\b(header|title|name|headline)\b', class_attr)

#we loop over both the Newest and New (Front) page
#We will loop over 1 to illustrate, when scraping this would be 1000.
for v in range(1):
    time.sleep(300)
    #Define the date now
    current_date = dt.datetime.now()
    #Go through the already made files
    pattern = re.compile(r"New_file(\d+)")
    files = os.listdir('.')

    # Initialize the variable to keep track of the highest number
    max_number = -1

    # Iterate over the files and find the highest number
    for file in files:
        match = pattern.match(file)
        if match:
            number = int(match.group(1))
            if number > max_number:
                max_number = number
    print(max_number)
    g = max_number
    
    #Reset all variables    
    Website = requests.get("https://news.ycombinator.com/newest")
    soup = BeautifulSoup(Website.text, 'html.parser') 
    #Make an all_data list
    all_data = []
    #Take the saved data in a list to all_data
    with open(f"New_file{g}.json", "r") as save_file:  
        all_data = json.load(save_file)

    #start off with both conditions being True
    cond1 = True
    cond2 = True

    #NEWEST
    #This condition will change when we reach 12h the first loop, 
    #or until we reach a duplicate in the second or until we dont find any more tables (website error).
    while cond1 and cond2:
        time.sleep(10)
        table = soup.find('table')

        if table:
            for index, (title_soup,posted_at_new,user_new,votes_new,comment_soup) in enumerate(zip(table.find_all('span', attrs={'class':'titleline'}),
                table.find_all('span', attrs={'class':'age'}), table.find_all('a', attrs={'class':'hnuser'}),
                table.find_all('span', attrs={'class':'score'}), table.find_all('td', attrs={'class':'subtext'}))):
                #Put all values back to Null
                title = None
                url = None
                domain = None
                posted_at = None
                user = None
                votes = None 
                comment = None
                source_title = None
                source_text = None
                cont_cond = False
                break_cond = False

                #Define posted_at first, so we can skip the whole loop if not relevant
                posted_at = age_to_time(posted_at_new.get_text())
                if 359 < posted_at < 721:
                    pass
                elif posted_at < 360 :
                    print("Skipped") #Interesting to see whether the entry was skipped or not
                    continue
                else: 
                    cond2 = False
                    print("False2") #This may only happen the first time through the loop
                    break
                
                #The date the post was gathered
                post_date = current_date - timedelta(minutes = posted_at)
                posted_at_date = post_date.strftime('%Y-%m-%d %H:%M:%S')
                
                #Find the Title
                match = re.match(r"^(.*?) (?:\[(?:video|.+?)\] )?\(([a-zA-Z].*?)\)", title_soup.text)
                if match:
                    title, domain_new = match.groups()
                else:
                    title = title_soup.text
                    domain_new = None

                #Define the User and Vote
                user = user_new.get_text()
                votes = votes_new.get_text()

                #go through the file, and if it is a dupe stop the whole while loop.
                for item in all_data:
                    if item['Title'] == title and item['User'] == user:
                        #If one or multiple posts just appear while going through them they might get 
                        #displaced from one page to another, if something like this is not in function 
                        #it will just stop scanning. This is to prevent that.
                        if index in (0, 1, 2): 
                            print("False1.1")
                            cont_cond = True
                            break #just breaks the for loop we are in now
                        else:
                            print("False1.2")
                            cond1 = False
                            break_cond = True
                            break #just breaks the for loop we are in now
                    else:
                        pass

                #If it is an index 0, 1 or 2 (the few first posts), there is still a possibility
                #too much time passed scraping and the posts went to the next page and the other
                #posts on the page are relevant, if it is not the first few, we reached a duplicate,
                #so we have to break the whole loop and the scraping of Newest will stop.
                if cont_cond == True:
                    continue
                if break_cond == True:
                    break

                #Find the Url
                title_link = title_soup.find('a')
                if title_link:
                    url = title_link['href']
                try: 
                    url_req = requests.get(url)
                    if url_req.status_code == 200:
                        parsed = urlparse(url)
                        domain = parsed.netloc
                except:
                    url = None

                #If the url didn't get the domain, we can assign it if Hackernews described it.
                if domain == None:
                    domain = domain_new

                #for multiple comments
                mat = re.search(r'\bcomments\b', comment_soup.text.lower())
                #for 1 comment
                mat1 = re.search(r'\bcomment\b', comment_soup.text.lower())
                if mat:   
                    #Get the amount of comments by mirroring and starting to count backwards (by mirroring it), 
                    #in which case it is the first number.
                    comment_number = re.search(r'\d+', comment_soup.text[::-1])
                    number = int(comment_number.group())
                    #Mirror the number it back
                    comment = str(number)[::-1]
                #There were some slight complications with comment(s) so I decided to make 2 of them.
                #1 with and 1 without s.
                elif mat1: 
                    comment = '1'
                else:
                    comment = '0' 


                #For Scraping
                try: 
                    Website_url = requests.get(url)
                    soup = BeautifulSoup(Website_url.text, 'html.parser') 
                    #Source title scraping
                    source_title_new = soup.find('h1', class_=contains_desired_class)
                    if source_title_new == None: 
                        source_title_new = soup.find('h1')
                        if source_title_new == None:
                            source_title_new = soup.find('h3')
                    source_title = source_title_new.get_text(strip=True) 
                    #Source text scraping 
                    source_text_temp = soup.find_all('p')
                    source_text = source_text_temp[0].get_text(strip=True)
                except:
                    pass

                data = {
                    "Title" : title, 
                    "URL" : url,
                    "Domain" : domain,
                    "Posted_at": posted_at_date,
                    "User" : user, 
                    "Votes" : votes,
                    "Comments" : comment,
                    "Source_title" : source_title,
                    "Source_text" : source_text
                }

                #add the file, it went through because the posted_at was right and title/user were not dupes.
                all_data.append(data) 
                print("Appended1: " + title + " & " + str(posted_at_date)) #interesting to see if the entry was recorded and when
        #If there is a problem with the table/website, too many requests probably
        else:
            print("ERROR")
            break

        more = table.find('a', attrs={'class':'morelink'}) 
        hrefmore = more.get('href')
        print(hrefmore)

        Website = requests.get("https://news.ycombinator.com/" + hrefmore)
        soup = BeautifulSoup(Website.text, 'html.parser') 

    #sort on age from youngest to oldest, because then we will find the duplicates the fastest
    sorted_data = sorted(all_data, key=lambda directory: directory['Posted_at'])
    with open(f"New_file{g+1}.json", "w") as save_file:  
            json.dump(sorted_data, save_file, indent = 9)


    # Now we go to the FrontPage
    print("---------------")

    # Appoint the website (Frontpage)
    Website1 = requests.get("https://news.ycombinator.com/news")
    soup = BeautifulSoup(Website1.text, 'html.parser')    

    #Make an all_data_front list
    all_data_front = []
    #Take the saved data in a list to all_data_front
    with open(f"Front_File{g}.json", "r") as save_file:  
        all_data_front = json.load(save_file)    

    #FRONTPAGE
    #After 5 pages, there will be nothing not 1d+ old, I took 8 pages to be sure.
    for i in range(8):
        time.sleep(10)
        table = soup.find('table')
        if table:
            for index, (title_soup,posted_at_new,user_new,votes_new,comment_soup) in enumerate(zip(table.find_all('span', attrs={'class':'titleline'}),
                table.find_all('span', attrs={'class':'age'}), table.find_all('a', attrs={'class':'hnuser'}),
                table.find_all('span', attrs={'class':'score'}), table.find_all('td', attrs={'class':'subtext'}))):
                #Put all values back to Null
                title = None
                url = None
                domain = None
                posted_at = None
                user = None
                votes = None 
                comment = None
                source_title = None
                source_text = None
                continue_cond = False

                #Define posted_at first, so we can skip the whole loop if not relevant
                posted_at = age_to_time(posted_at_new.get_text())
                if posted_at < 721:
                    pass
                else:
                    print("Skipped") #Interesting to see whether the entry was skipped or not
                    continue #Skip this particular post bc it is not relevant
                    
                #The date the post was gathered
                post_date = current_date - timedelta(minutes = posted_at)
                posted_at_date = post_date.strftime('%Y-%m-%d %H:%M:%S')
                
                #Find the Title
                match = re.match(r"^(.*?) (?:\[(?:video|.+?)\] )?\(([a-zA-Z].*?)\)", title_soup.text)
                if match:
                    title, domain_new = match.groups()
                else:
                    title = title_soup.text
                    domain_new = None

                #Define the User and Vote
                user = user_new.get_text()
                votes = votes_new.get_text()

                #go through the file, and if it is a dupe stop the whole while loop.
                for item in all_data_front:
                    if item['Title'] == title and item['User'] == user:
                        #print("Dupe")
                        continue_cond = True
                        break

                #I do it this way, otherwise it would only break the nested loop, not the table one.
                #If it is an index 0, 1 or 2 (the first items), there is still a possibility
                #too much time passed scraping and the post went to the next page
                #but the other posts on the page are relevant
                if continue_cond == True:
                    continue

                #Find the Url
                title_link = title_soup.find('a')
                if title_link:
                    url = title_link['href']
                try: 
                    url_req = requests.get(url)
                    if url_req.status_code == 200:
                        parsed = urlparse(url)
                        domain = parsed.netloc
                except:
                    url = None

                #If the url didn't get the domain, we can assign it if Hackernews described it.
                if domain == None:
                    domain = domain_new

                #for multiple comments
                mat = re.search(r'\bcomments\b', comment_soup.text.lower())
                #for 1 comment
                mat1 = re.search(r'\bcomment\b', comment_soup.text.lower())
                if mat:   
                    #Get the amount of comments by mirroring and starting to count backwards (by mirroring it), 
                    #in which case it is the first number.
                    comment_number = re.search(r'\d+', comment_soup.text[::-1])
                    number = int(comment_number.group())
                    #Mirror the number it back
                    comment = str(number)[::-1]
                #There were some slight complications with comment(s) so I decided to make 2 of them.
                #1 with and 1 without s.
                elif mat1: 
                    comment = '1'
                else:
                    comment = '0' 


                #For Scraping
                try: 
                    Website_url = requests.get(url)
                    soup = BeautifulSoup(Website_url.text, 'html.parser') 
                    #Source title scraping
                    source_title_new = soup.find('h1', class_=contains_desired_class)
                    if source_title_new == None: 
                        source_title_new = soup.find('h1')
                        if source_title_new == None:
                            source_title_new = soup.find('h3')
                    #Source text scraping 
                    source_text_temp = soup.find_all('p')
                    source_text = source_text_temp[0].get_text(strip=True)
                    #for source_text in soup.findAll('p'):
                        #source_text_temp = []
                        #append to temp 
                        #print(source_text)
                        #source_text_temp.append(source_text.get_text(strip = True))
                    #source_text = ' '.join(source_text_temp)
                    #Now we add them to the lists  
                    source_title = source_title_new.get_text(strip=True)       
                except:
                    pass

                data = {
                   "Title" : title, 
                    "URL" : url,
                    "Domain" : domain,
                    "Posted_at": posted_at_date,
                    "User" : user, 
                    "Votes" : votes,
                    "Comments" : comment,
                    "Source_title" : source_title,
                    "Source_text" : source_text
                }

                #add the file, it went through because the posted_at was right and title/user were not dupes.
                all_data_front.append(data) 
                print("Appended2: " + title + " & " + str(posted_at_date)) #interesting to see if the entry was recorded and when
        #If there is a problem with the table/website, too many requests probably
        else:
            print("ERROR")
            break
        #If there is any problem getting to the next website, we just get the same data, 
        #Meaning we will get a cond1 False
        more = table.find('a', attrs={'class':'morelink'}) 
        hrefmore = more.get('href')
        print(hrefmore)

        Website1 = requests.get("https://news.ycombinator.com/" + hrefmore)
        soup = BeautifulSoup(Website1.text, 'html.parser') 

    #Just save the data, sorting on age will do nothing here
    with open(f"Front_File{g+1}.json", "w") as save_file:  
        json.dump(all_data_front, save_file, indent = 9)    
    print("Ended")

646
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
newest?next=40481209&n=31
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
newest?next=40480799&n=61
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
newest?next=40480359&n=91
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skippe

Part 2: Preprocessing

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, when, row_number, lit
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.sql.window import Window
from pyspark.ml import PipelineModel, Pipeline
from pyspark.streaming import StreamingContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
#Creating a Spark Session
spark = SparkSession.builder.appName("HackerNews").getOrCreate()

json_front = 'Front_file641.json'
json_newest = 'New_file641.json'

df_front = pd.read_json(json_front)
df_new = pd.read_json(json_newest)

#We make a label called frontpage to differentiate the posts
df_front['frontpage'] = 1
df_new['frontpage'] = 0

#put the data together
df = pd.concat([df_front, df_new])
df.reset_index(drop=True, inplace=True)

#Sort based on oldest as is required
df = df.sort_values(by='Posted_at')

#See that every value of 'Votes' has 'point' in the string
df = df[df['Votes'].str.contains('point')]

#df.dropna(inplace = True)
columns = df.columns.difference(['Source_text'])
df.dropna(subset=columns, inplace= True, how='any')

psdf = spark.createDataFrame(df)

#We make the votes into integers
def points_to_int(i):
    if 'point' in i:
        return int(i.split()[0])
    else:
        return None

#Make a UDF
points_to_int_udf = udf(points_to_int, IntegerType())

#Put them in the Votes column
psdf = psdf.withColumn('Votes', points_to_int_udf(col('Votes')))

#put the titles in lower case, to match the model later on.
def rename_columns_to_lowercase(df):
    for title in df.columns:
        df = df.withColumnRenamed(title, title[0].lower() + title[1:])
    return df

psdf = rename_columns_to_lowercase(psdf)
psdf = psdf.withColumnRenamed('uRL', "url")

psdf.show()
psdf.printSchema

+--------------------+--------------------+-------------------+-------------------+--------------+-----+--------+--------------------+--------------------+---------+
|               title|                 url|             domain|          posted_at|          user|votes|comments|        source_title|         source_text|frontpage|
+--------------------+--------------------+-------------------+-------------------+--------------+-----+--------+--------------------+--------------------+---------+
|Developers' Perce...|https://arxiv.org...|          arxiv.org|2024-05-21 14:22:39|        belter|    2|       0|Title:Developers'...|Help|Advanced Search|        0|
|Scale AI raises $...|https://techcrunc...|     techcrunch.com|2024-05-21 14:22:39|danielmorozoff|    3|       0|Data-labeling sta...|             Comment|        0|
|Increasing LLM Ac...|https://arxiv.org...|          arxiv.org|2024-05-21 14:22:39|        belter|    1|       0|Title:Increasing ...|Help|Advanced Search|        0|
|Com

<bound method DataFrame.printSchema of DataFrame[title: string, url: string, domain: string, posted_at: timestamp, user: string, votes: int, comments: bigint, source_title: string, source_text: string, frontpage: bigint]>

In [5]:
string_columns = ["title", "url", "domain", "user", "source_title"]
numeric_columns = ["votes", "comments"]

for col_name in string_columns:
   psdf = psdf.withColumn(col_name, when(col(col_name) == "", None).otherwise(col(col_name)))
psdf.na.drop()

indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid='keep') for col in string_columns]
encoders = [OneHotEncoder(inputCol=col + "_index", outputCol=col + "_encoded") for col in string_columns]

feature_columns = [col + "_encoded" for col in string_columns] + numeric_columns

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

lr = LogisticRegression(featuresCol="features", labelCol="frontpage")

pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])

Part 3: Model Training

In [6]:
#Make a train and test-set
train_data, test_data = psdf.randomSplit([0.8,0.2], seed = 54321)

model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="frontpage")
accuracy = evaluator.evaluate(predictions)

model.save('LRmodel')

In [7]:
# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.7639509255105427


Part 3: Spark Streaming

In [8]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [9]:
import random
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [10]:
globals()['models_loaded'] = False
globals()['my_model'] = None

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = PipelineModel.load("LRmodel")
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model (uncomment below):
    
    df_result = globals()['my_model'].transform(df)
    df_result.show()

In [11]:
ssc = StreamingContext(sc, 10)
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)
ssc_t = StreamingThread(ssc)
ssc_t.start()

C:\Users\alexf\OneDrive\Bureaublad\spark\spark-3.5.1-bin-hadoop3\python\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


========= 2024-05-26 13:20:50 =========
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+
|     aid|comments|              domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|       user|votes|
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------+-----+
|40478628|       0|thebignewsletter.com|    false|2024-05-25 23:30:53|Antitrust Enforce...|Antitrust Enforce...|Antitrust Enforce...|https://www.thebi...|toomuchtodo|    2|
|40478648|       0|      independent.ie|    false|2024-05-25 23:35:46|The future of Big...|The future of Big...|The future of Big...|https://www.indep...|ewgfdgdfgdf|    1|
|40478665|       0|         youtube.com|    false|2024-05-25 23:39:47|How are Microchip...|How 

In [12]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----+
|     aid|comments|              domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|          user|votes|
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----+
|40478721|       0|github.com/n64recomp|    false|2024-05-25 23:53:13|GitHub - N64Recom...|GitHub - N64Recom...|     N64: Recompiled|https://github.co...|      linksbro|    1|
|40478733|       0|        casetext.com|    false|2024-05-25 23:55:15|Just a moment...\...|    Just a moment...|Casetext: AI Lega...|https://casetext....|handfuloflight|    2|
|40478735|       0|github.com/rochus...|    false|2024-05-25 23:55:3